In [1]:
import findspark
findspark = findspark.init()

import os
import datetime
import time
import pandas as pd
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql import Row
import pyspark.sql.types as T


/Users/aravind_jarpala/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/aravind_jarpala/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:

spark = SparkSession.builder. \
        appName('FileFormatComparison'). \
        config('spark.driver.host', 'localhost'). \
        config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.5.1"). \
        getOrCreate()

26/01/11 14:39:39 WARN Utils: Your hostname, Aravinds-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.102 instead (on interface en0)
26/01/11 14:39:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/aravind_jarpala/opt/anaconda3/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/aravind_jarpala/.ivy2/cache
The jars for the packages stored in: /Users/aravind_jarpala/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-85b5801e-9cec-424a-869b-b6e429497e92;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.5.1 in central
	found org.tukaani#xz;1.9 in central
:: resolution report :: resolve 133ms :: artifacts dl 3ms
	:: modules in use:
	org.apache.spark#spark-avro_2.12;3.5.1 from central in [default]
	org.tukaani#xz;1.9 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	------------------------------------------------

In [3]:
num_rows = 10000000

df = spark.range(0, num_rows)

for i in range(1, 10):
    if i % 2 == 0:
        df = df.withColumn(f'int_col_{i}', (rand() * 100).cast(T.IntegerType()))
    else:
        df = df.withColumn(f'string_col_{i}', (rand() * num_rows).cast(T.IntegerType()).cast('string'))

df.count()

10000000

In [4]:
df.show(10, truncate=False)

+---+------------+---------+------------+---------+------------+---------+------------+---------+------------+
|id |string_col_1|int_col_2|string_col_3|int_col_4|string_col_5|int_col_6|string_col_7|int_col_8|string_col_9|
+---+------------+---------+------------+---------+------------+---------+------------+---------+------------+
|0  |9367135     |37       |7262903     |37       |6255236     |84       |9020969     |69       |2006516     |
|1  |4271706     |84       |1007273     |62       |8752255     |18       |544379      |13       |6266682     |
|2  |3183130     |64       |3655952     |83       |3239296     |26       |2180526     |67       |1117432     |
|3  |134850      |49       |5871651     |95       |3254110     |92       |1189300     |34       |9033531     |
|4  |4146443     |80       |285856      |9        |7002961     |78       |6550380     |55       |6680912     |
|5  |2445081     |75       |5145765     |98       |8589070     |61       |1524248     |19       |453510      |
|

In [6]:
base_path = f"file-formats/tmp"

#write as different file formats
df.write.mode("overwrite").csv(f"{base_path}/csv")
df.write.mode("overwrite").parquet(f"{base_path}/parquet")
df.write.mode("overwrite").format("avro").save(f"{base_path}/avro")

26/01/11 14:40:48 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


##### File Format Size

In [8]:
!du -sh file-formats/tmp/*

462M	file-formats/tmp/avro
649M	file-formats/tmp/csv
420M	file-formats/tmp/parquet


#### Columnar Pruning